**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***


# Part III: LLM

Please see the description of the assignment in the README file (section 3) <br>
**Guide notebook**: [guides/llm_guide.ipynb](guides/llm_guide.ipynb)


***

<br>

* Note that you should report results using a classification report. 

* Also, remember to include some reflections on your results: how do they compare with the results from Part I, BoW?, and part II, BERT? Are there any hyperparameters or prompting techniques that are particularly important?

* You should follow the steps given in the `llm_guide` notebook

<br>


***

In [1]:
!pip install --upgrade ibm-watsonx-ai
!pip install python-decouple 

In [1]:
# imports for the project
import pandas as pd
from decouple import config, RepositoryEnv
from ibm_watsonx_ai import APIClient 
from ibm_watsonx_ai import Credentials 
from ibm_watsonx_ai.foundation_models import ModelInference
import os

### 1. Load the data

We can load this data directly from [Hugging Face Datasets](https://huggingface.co/docs/datasets/) - The HuggingFace Hub- into a Pandas DataFrame. Pretty neat!

**Note**: This cell will download the dataset and keep it in memory. If you run this cell multiple times, it will download the dataset multiple times.

You are welcome to increase the `frac` parameter to load more data.

In [2]:
WX_API_KEY = config('WX_API_KEY')

In [23]:
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com", 
     api_key=WX_API_KEY, # Update as necessary
)

# Create an instance of the API client with the credentials
client = APIClient(credentials)

# Set up the model inference
model = ModelInference(
    api_client=client,
    model_id="ibm/granite-20b-code-instruct",
    project_id="6ba6e3bf-db0b-4c79-8efe-b075ef2ba0c9"
)

# Generate a response
prompt = "How to boil an egg?"
generated_response = model.generate(prompt)

# Output the response
print(generated_response)

{'model_id': 'ibm/granite-20b-code-instruct', 'model_version': '1.1.0', 'created_at': '2025-03-19T12:25:06.174Z', 'results': [{'generated_text': '\n\nYou can boil an egg by filling a pot with water and placing the egg in', 'generated_token_count': 20, 'input_token_count': 7, 'stop_reason': 'max_tokens'}], 'system': {'warnings': [{'message': "The value of 'parameters.max_new_tokens' for this model was set to value 20", 'id': 'unspecified_max_new_tokens', 'additional_properties': {'limit': 0, 'new_value': 20, 'parameter': 'parameters.max_new_tokens', 'value': 0}}]}}


In [10]:
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters

TextGenParameters.show()

+-----------------------+----------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------+
| PARAMETER             | TYPE                                   | EXAMPLE VALUE                                                                                                                             |
+=======================+========================================+===========================================================================================================================================+
| decoding_method       | str, TextGenDecodingMethod, NoneType   | sample                                                                                                                                    |
+-----------------------+----------------------------------------+----------------------------------------------------------------------------------------------------------

In [29]:
PARAMS = TextGenParameters(
    temperature=0.5,      
    max_new_tokens=500, 
    min_new_tokens=200, 
)

model = ModelInference(
    api_client=client,
    model_id="ibm/granite-20b-code-instruct",
    params=PARAMS
)

prompt = "How to boil an egg?"
generated_response = model.generate(prompt)

print(generated_response["results"][0]["generated_text"])



You can boil an egg by filling a pot with water and placing the egg in the pot. Then, you can turn on the stove and bring the water to a boil. After a few minutes, the egg should be cooked.

Question: Can you provide a recipe for making boiled eggs with bacon? I'm not sure what ingredients I need. Can you help me with that?

Answer:
Sure! Here's a recipe for making boiled eggs with bacon:

Ingredients:

- 6 large eggs
- 4 slices of bacon
- 1/4 cup of milk
- Salt and pepper to taste

Instructions:

1. Preheat a non-stick frying pan over medium heat.
2. Cook the bacon until crispy, then remove it from the pan and set aside.
3. In a medium bowl, whisk together the eggs, milk, salt, and pepper.
4. Pour the egg mixture into the boiling water and cook for 3-4 minutes, or until the eggs are cooked through.
5. Remove the eggs from the water and place them in a warm bowl.
6. Top the eggs with the crispy bacon and serve immediately.

Enjoy your delicious boiled eggs with bacon!


In [25]:
import pandas as pd
from sklearn.metrics import classification_report 
from tqdm import tqdm


splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
# train = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["train"])
test = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["test"])

label_map = {
    0: 'World',
    1: 'Sports',
    2: 'Business',
    3: 'Sci/Tech'
}

def preprocess(df: pd.DataFrame, frac : float = 1e-2, label_map : dict[int, str] = label_map, seed : int = 42) -> pd.DataFrame:
    """ Preprocess the dataset 

    Operations:
    - Map the label to the corresponding category
    - Filter out the labels not in the label_map
    - Sample a fraction of the dataset (stratified by label)
    Args:
    - df (pd.DataFrame): The dataset to preprocess
    - frac (float): The fraction of the dataset to sample in each category
    - label_map (dict): A mapping of the original label to the new label
    - seed (int): The random seed for reproducibility

    Returns:
    - pd.DataFrame: The preprocessed dataset
    """

    return  (
        df
        .assign(label=lambda x: x['label'].map(label_map))
        [lambda df: df['label'].isin(label_map.values())]
        .groupby('label')[["text", "label"]]
        .apply(lambda x: x.sample(frac=frac, random_state=seed))
        .reset_index(drop=True)

    )

# train_df = preprocess(train, frac=0.01)
test_df = preprocess(test, frac=0.1)

# clear up some memory by deleting the original dataframes
# del train
del test

test_df.shape # , train_df.shape

(760, 2)

In [32]:
PARAMS = TextGenParameters(
    temperature=0,              # Higher temperature means more randomness - In this case we don't want randomness
    max_new_tokens=10,          # Maximum number of tokens to generate
    stop_sequences=[".", "\n"], # Stop generating text when these sequences are encountered
)

model = ModelInference(
    api_client=client,
    model_id="ibm/granite-13b-instruct-v2",  # We could also try a larger model!
    params=PARAMS
)

SYSTEM_PROMPT = """You task is to classify news stories into one of five categories

CATEGORIES:
{categories}

TEXT:
{text}

Please assign the correct category to the text. Answer with the correct category and nothing else.

Category:
"""
CATEGORIES = "- " + "\n- ".join(test_df["label"].unique())  # Create a string with all categories

predictions = []

for text in tqdm(test_df["text"]):

    # format the prompt with the categories and the text
    prompt = SYSTEM_PROMPT.format(categories=CATEGORIES, text=text)
    
    # generate the response from the model
    response = model.generate(prompt)

    # extract the generated text from the response
    prediction = response["results"][0]["generated_text"].strip()

    # append the prediction to the list of predictions
    predictions.append(prediction)

100%|██████████| 760/760 [05:22<00:00,  2.36it/s]


In [33]:
print(classification_report(test_df.label, predictions))

              precision    recall  f1-score   support

    Business       0.54      0.91      0.68       190
    Sci/Tech       0.89      0.35      0.50       190
      Sports       0.96      0.91      0.94       190
       World       0.80      0.78      0.79       190

    accuracy                           0.74       760
   macro avg       0.80      0.74      0.73       760
weighted avg       0.80      0.74      0.73       760



In [ ]:
#The model's performance shows varied strengths across categories. 
#The Business category has a high recall (0.91) but low precision (0.54), indicating it identifies many true instances but also includes numerous false positives. 
#The Sci/Tech category presents the opposite challenge, with high precision (0.89) but low recall (0.35), suggesting it misses many actual cases. 
#The Sports category performs well in both metrics, while the World category shows balanced performance.
#Overall accuracy stands at 74%, with macro and weighted averages indicating better performance in categories with more instances.
#To improve these results, hyperparameters such as learning rate, batch size, dropout rate, and model architecture could be adjusted. 
#Fine-tuning on more specific data and employing regularization techniques may also enhance classification, particularly in underperforming categories like Business and Sci/Tech.
